In [1]:
import dores
import pandas as pd
pd.set_option('max_colwidth', -1)

In [2]:
st_brieuc = dores.DiscreteEventSimulation()
entities = st_brieuc.create_entities()
st_brieuc.start_simulation()
st_brieuc.get_event_log()

,Timestamp,ID,ActivityDescription,ActivityState,ComponentsInStock,ComponentsOnboard,ComponentsInstalled
0,2021-01-01 00:00:00,AEOLUS,Navigate from CHERBOURG-OCTEVILLE to SAINT BRIEUC OFFSHORE WIND FARM,INITIATED,50,0,0
1,2021-01-01 01:00:00,AEOLUS,Navigate from CHERBOURG-OCTEVILLE to SAINT BRIEUC OFFSHORE WIND FARM,INITIATED,50,0,0
2,2021-01-01 01:00:00,AEOLUS,Navigate from SAINT BRIEUC OFFSHORE WIND FARM to CHERBOURG-OCTEVILLE,INITIATED,50,0,0
3,2021-01-01 02:00:00,AEOLUS,Navigate from SAINT BRIEUC OFFSHORE WIND FARM to CHERBOURG-OCTEVILLE,INITIATED,50,0,0
